## GOAL
Understand the X gate (bit flip). What can we do with this gate?


In [ ]:
# Import Qiskit tools.
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram

# Build a 1-qubit circuit with X.
qc_x = QuantumCircuit(1, 1)
qc_x.x(0)
qc_x.measure(0, 0)

# Run on local sampler.
sampler = Sampler()
res_x = sampler.run([qc_x], shots=1024).result()
counts_x = {k: int(v * 1024) for k, v in res_x.quasi_dists[0].items()}
print("X counts:", counts_x)
plot_histogram(counts_x)


## EXERCISE
Predict the result before running. Does X always give |1>?


## GOAL
Understand the H gate (creates superposition). What can we do with this gate?


In [ ]:
# Build a 1-qubit circuit with H.
qc_h = QuantumCircuit(1, 1)
qc_h.h(0)
qc_h.measure(0, 0)

# Run on local sampler.
res_h = sampler.run([qc_h], shots=1024).result()
counts_h = {k: int(v * 1024) for k, v in res_h.quasi_dists[0].items()}
print("H counts:", counts_h)
plot_histogram(counts_h)


## EXERCISE
Predict the result of H then H (two H gates).


## GOAL
Understand the Z gate (phase flip). What can we do with this gate?


In [ ]:
# Build a 1-qubit circuit with Z and H to reveal phase.
qc_z = QuantumCircuit(1, 1)
qc_z.h(0)
qc_z.z(0)
qc_z.h(0)
qc_z.measure(0, 0)

# Run on local sampler.
res_z = sampler.run([qc_z], shots=1024).result()
counts_z = {k: int(v * 1024) for k, v in res_z.quasi_dists[0].items()}
print("Z counts:", counts_z)
plot_histogram(counts_z)


## EXERCISE
Why is Z invisible on |0> but visible after H?


## GOAL
Understand the CX gate (controlled X). What can we do with this gate?


In [ ]:
# Build a 2-qubit circuit with CX.
qc_cx = QuantumCircuit(2, 2)
qc_cx.x(0)
qc_cx.cx(0, 1)
qc_cx.measure([0, 1], [0, 1])

# Run on local sampler.
res_cx = sampler.run([qc_cx], shots=1024).result()
counts_cx = {k: int(v * 1024) for k, v in res_cx.quasi_dists[0].items()}
print("CX counts:", counts_cx)
plot_histogram(counts_cx)


## EXERCISE
Predict counts if the control qubit stays in |0>.


## GOAL
Encrypt and decrypt a single qubit using X^a Z^b (QOTP).


In [ ]:
# Import random and statevector.
import random
from qiskit.quantum_info import Statevector

# Choose random key bits.
a = random.randint(0, 1)
b = random.randint(0, 1)
print("Key bits a, b:", a, b)

# Prepare |+> state.
qc_qotp = QuantumCircuit(1)
qc_qotp.h(0)
original = Statevector.from_instruction(qc_qotp)

# Encrypt: apply X^a then Z^b.
if a == 1:
    qc_qotp.x(0)
if b == 1:
    qc_qotp.z(0)

# Decrypt: apply Z^b then X^a.
if b == 1:
    qc_qotp.z(0)
if a == 1:
    qc_qotp.x(0)

# Compare final state to original.
final = Statevector.from_instruction(qc_qotp)
fidelity = abs(original.data.conjugate().dot(final.data)) ** 2
print("Fidelity:", fidelity)


## EXERCISE
Try all four keys (a,b) and confirm decryption works. Then switch the input to |1>.
